**Quel est l'intérêt de coder une classe Python ?** J'ai eu beaucoup de mal à le trouver quand j'ai commencé à apprendre le langage, mais une [formation d'analyste de données à la Wild Code School](https://www.wildcodeschool.com/fr-FR/formations/formation-data-analyst) est depuis passée par là. En reformulant la question par *quand devrait-on coder une classe Python*, je répondrai tout simplement quand :

* on doit **répéter des tâches rébarbatives** et **automatisables par des fonctions**
* ces fonctions seraient plus efficaces **si on les appliquait à un objet informatique**. Et heureux hasard, Python est un langage multi-paradigme **compatible avec la Programmation orientée objet** (POO)

Pour mes dernières piges, j'ai dû beaucoup recycler un script de scraping, notamment pour récupérer depuis Wikipedia les codes INSEE de différentes communes [comme celles composant l'Eurométropole de Strasbourg](https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg). Ces manip sont toujours réalisées avec les modules suivants :

In [1]:
import re, requests
import pandas as pd
from bs4 import BeautifulSoup

Et, il faut le dire, il y a une fonction intégrée dans pandas qui permet de transformer en DataFrame un tableau HTML de ce type :

![table_html](images/table_html.jpg)

In [2]:
table_eurom=pd.read_html('https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg', match="Liste des communes de l’intercommunalité",decimal=',')
table_eurom[0].head()

,Nom,Code Insee,Gentilé,Superficie (km2),Population (dernière pop. légale),Densité (hab./km2)
0,Strasbourg(siège),67482,Strasbourgeois,7826,287 228 (2019),3 670
1,Achenheim,67001,Achenheimois,603,2 338 (2019),388
2,Bischheim,67043,Bischheimois,441,17 353 (2019),3 935
3,Blaesheim,67049,Blaesheimois,996,1 301 (2019),131
4,Breuschwickersheim,67065,Breuschwickersheimois,506,1 313 (2019),259


Cela semble pas mal à première vue, mais il y a quand même quelques scories à relever :

* les nombres à décimales de la colonne "Superficie (km2)" ont été mal interprétés, puisque les virgules ont disparu (alors qu'on a bien précisé dans l'appel de fonction le séparateur ",")
* les colonnes de population et de densité ont des espaces, leurs cellules sont donc interprétées autrement que comme des chiffres. La preuve ?

In [3]:
table_eurom["Population (dernière pop. légale)"].sum()

TypeError: ignored

Cela pourrait se corriger directement dans la DataFrame, mais gardons en tête un des principaux avantages du code : **pouvoir personnaliser selon ses propres besoins**. Ici en convertissant ce qui peut l'être en nombres entiers ou à décimales.

On va se servir de ce simple prétexte afin d'explorer l'intérêt des classes Python. Voilà le cahier des charges de celle que j'ai nommée Wikidaper :
* **ne considérer que les pages issues de Wikipedia**, dont l'URL est conforme à celles copiées/collées depuis un navigateur
* repérer **les tableaux HTML avec des données classables** (astuce : ils ont toujours une classe qui contient les termes "wikitable" et "sortable" et les ranger dans une liste
* pour un tableau interrogé dans la liste (si elle n'est pas vide), créer une DataFrame pandas dans laquelle **les valeurs chiffrées sont bien converties en chiffres entiers ou à décimales**

On essayera de ne pas trop forcer le côté *quick and dirty*. Même si ce caclepin n'a aucune autre prétention que de **constituer une porte d'entrée aux classes Python**, j'espère que le code sera assez soigné pour être réexploité par qui le souhaite.

Si vous ne voulez pas vous embêter avec l'ensemble des explications, le code source de l'ensemble est [dans ce repo Github](https://github.com/raphadasilva/wikidaper).

# Codage de la classe

Voici une première déclaration de classe :

In [4]:
class Wikidaper:
    """
        Cette classe sert à aspirer les tableaux HTML qui contiennent des données classables dans une page Wikipedia.
        Les données chiffrées sont converties en entiers ou nombres à décimales et les tableaux restitués en DataFrames.
    """
    def __init__(self,url:str):
        """
            On initialise avec une URL renseignée par l'utilisateur, de préférence copiée/collée depuis le navigateura
        """
        self.url=url

Remarquez **la présence du mot-clé self**, qui confirme qu'on modifie une instance de la classe. C'est l'équivalent exact du this en Javascript.

Pour l'instant, on a juste créé une fonction d'initialisation de la classe Wikidaper. Elle peut dès lors être affectée à une variable, par exemple :

In [5]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Mulhouse")

Cette variable a son propre type :

In [6]:
type(test_wiki)

__main__.Wikidaper

Ainsi que des attributs, que l'on peut afficher sous la forme d'un dictionnaire :

In [7]:
test_wiki.__dict__

{'url': 'https://fr.wikipedia.org/wiki/Mulhouse'}

**Une initialisation seule ne sert pas à grand chose**, et il faudra sans doute **améliorer Wikidaper avec quelques fonctions**. Par exemple si on réinitialise test_wiki comme suit :

In [8]:
test_wiki=Wikidaper("https://www.rue89strasbourg.com")
test_wiki.__dict__

{'url': 'https://www.rue89strasbourg.com'}

Il n'y a aucun message d'erreur, alors qu'on a interrogé un site autre que Wikipedia. Comment régler ce souci ?

On va utiliser une expression régulière, autrement dit **une chaîne de caractères résumant plusieurs chaînes de caractères**. Par exemple :

^http\[s]{,1}:[/]{2}[a-z]{2,3}\.wikipedia\.org/wiki/[A-Za-z0-9]+[_A-Za-z0-9]*

Pas de panique, nous allons décomposer bloc par bloc :
* ^http signifie que la chaîne de caractères **doit obligatoirement commencer par http**. C'est une URL, c'est la moindre des choses !
* [s]{,1} stipule que **le caractère s doit apparaître une fois maximum**. Avec ce qui a été écrit précédemment, on râtisse déjà deux chaînes de caractères (http et https). Tout ce qui est différent passe à la trappe
* :[/]{2} signifie que **le reste de la chaîne doit être composé d'un : suivi strictement de deux /**
* [a-z]{2,3} stipule que l'on doit ensuite **n'avoir qu'entre deux et trois caractères alphabétiques minuscules**. Cela correspond aux noms de domaine linguistiques (fr, en ou cz, par exemple)
* \.wikipedia\.org est un cas un peu particulier : le point étant un caractère spécial dans la syntaxe d'une expression régulière (il se réfère à n'importe quel caractère), on doit **le précéder d'un \ pour bien préciser que c'est le seul caractère du point qui nous intéresse**. En prenant tout ce qui a été fait avant, on récupère déjà des URL telles que http://cz.wikipedia.org, https://fr.wikipedia.org, et **on exclut surtout toute chaîne ne correspondant pas à ce schéma**
* /wiki/[A-Za-z0-9]+[_A-Za-z0-9]\* signifie que la chaîne cible doit **inclure /wiki/ suivi par au moins une lettre alphabétique (majuscule ou minuscule) ou un chiffre**, qui doit apparaître une fois. **+ est un caractère spécial traduisible par "une fois ou plus"**, donc [A-Za-z0-9]+ peut aussi se dire "tout caractère dans l'éventail décrit entre crochets doit apparaître au moins une fois. Le reste, de l'expression [_A-Za-z0-9]\* peut se décrire comme "tout caractère décrit dans l'éventail entre crochets (même chose qu'avant, avec juste un éventuel tiret de séparation) peut apparaître zéro ou plus fois". *** est un autre caractère spécial** souvent rencontré dans les regex

Cette regex n'est pas parfaite (certains caractères spéciaux comme le + n'ont pas été intégrés) et pourrait sans doute être plus succincte, mais elle a le mérite d'être assez compréhensible et **de considérer la large majorité des URL susceptibles d'être intéressantes**.

**On n'insistera jamais assez sur l'intérêt des regex**, et le temps qu'elles peuvent faire gagner. Parmi les sites permettant de s'y frotter, je conseille vivement la section dédiée [sur HackerRank](https://www.hackerrank.com/).

Ce détour par les regex passé, reprenons la classe Wikidaper :

In [9]:
class Wikidaper:
    """
        Cette classe sert à aspirer les tableaux HTML qui contiennent des données classables dans une page Wikipedia.
        Les données chiffrées sont converties en entiers ou nombres à décimales et les tableaux restitués en DataFrames.
    """
    def __init__(self,url:str):
        """
            On initialise avec :
                - une URL renseignée par l'utilisateur, de préférence copiée/collée depuis le navigateur 
                - un booléen de validation de cette dernière
        """
        self.url=url
        self.valid_url=self.valide_url()
        
    def valide_url(self):
        """
            Cette fonction va vérifier si l'URL rentrée par l'utilisateur renvoie vers une page Wikipedia susceptible d'avoir des tableaux avec données classables.
        """
        result=False
        regex="^http[s]{,1}:[/]{2}[a-z]{2,}\.wikipedia\.org/wiki/[A-Za-z0-9]+[_A-Za-z0-9]*"
        if re.match(regex, self.url):
            try:
                requests.get(self.url)
                result=True
            except:
                print("Cette page Wikipedia ne répond pas")
        else:
            print("Attention, votre URL ne pointe pas vers une page Wikipedia valide")
        return result

Dans la fonction valide_url, **on interroge toujours l'instance définie par l'utilisateur via le self**. Cette fonction sert à ajouter le booléen valid_url dans la fonction d'initialisation, **un attribut qui indique si l'URL rentrée par l'utilisateur est une page Wikipedia interrogeable**.

On peut tester sa validité aussi bien avec une URL conforme...

In [10]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg")
test_wiki.__dict__

{'url': 'https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg',
 'valid_url': True}

...qu'avec une URL non prévue par l'expression régulière de référence :

In [11]:
test_wiki=Wikidaper("https://www.rue89lyon.fr")
test_wiki.__dict__

Attention, votre URL ne pointe pas vers une page Wikipedia valide


{'url': 'https://www.rue89lyon.fr', 'valid_url': False}

Et à présent ? À présent, on va **récolter les tableaux classables dans une page Wikipedia valide au schéma**. Cela s'entérine encore avec une expression régulière, cette fois-ci :

^wikitable.+sortable.*

Qu'on pourrait traduire par "doit commencer par wikitable, séparé d'au moins un caractère avant sortable, suivi par zéro ou plusieurs caractères". 

Cela permettra de récupérer tous les tableaux correspondants après avoir aspiré le code source de la page grâce à BeautifulSoup. Voici la nouvelle mise à jour de la classe :

In [12]:
class Wikidaper:
    """
        Cette classe sert à aspirer les tableaux HTML qui contiennent des données classables dans une page Wikipedia.
        Les données chiffrées sont converties en entiers ou nombres à décimales et les tableaux restitués en DataFrames.
    """
    def __init__(self,url:str):
        """
            On initialise avec :
                - une URL renseignée par l'utilisateur, de préférence copiée/collée depuis le navigateur
                - un booléen de validation de cette dernière
                - une liste composée des tableaux classables trouvés sur la page
        """
        self.url=url
        self.valid_url=self.valide_url()
        if self.valid_url:
            self.l_tableaux=self.recolte_tableaux()
        else:
            self.l_tableaux=[]
        
    def valide_url(self):
        """
            Cette fonction va vérifier si l'URL rentrée par l'utilisateur renvoie vers une page Wikipedia susceptible d'avoir des tableaux avec données classables.
        """
        result=False
        regex=r"^http[s]{,1}:[/]{2}[a-z]{2,}\.wikipedia\.org/wiki/[A-Za-z0-9]+[_A-Za-z0-9]*"
        if re.match(regex, self.url):
            try:
                requests.get(self.url)
                result=True
            except:
                print("Cette page Wikipedia ne répond pas")
        else:
            print("Attention, votre URL ne pointe pas vers une page Wikipedia valide")
        return result
    
    def recolte_tableaux(self):
        """
            Cette fonction retourne une liste des tableaux classables trouvables sur une page Wikipedia.
            Elle reste vide s'il n'y a rien.
        """
        l_tableaux=[]
        url=requests.get(self.url)
        soupe=BeautifulSoup(url.text, "lxml")
        regex=re.compile(r'^wikitable.+sortable.*')
        for t in soupe.find_all("table", {"class" : regex}):
            l_tableaux.append(t)
        return l_tableaux

On peut vérifier encore une fois la conformité de ce nouvel attribut l_tableaux, par exemple en affichant la longueur de la liste après interrogation d'une mauvaise page...

In [13]:
test_wiki=Wikidaper("https://rue89bordeaux.com/")
len(test_wiki.l_tableaux)

Attention, votre URL ne pointe pas vers une page Wikipedia valide


0

...ou bien avec une page Wikipedia enfermant un seul tableau classable...

In [14]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg")
len(test_wiki.l_tableaux)

1

...voire plus !

In [15]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es")
len(test_wiki.l_tableaux)

2

Le code source confirme que notre expression régulière a aussi bien récupéré une classe "wikitable sortable mw-collapsible centre alternance" que "wikitable sortable", **tout en excluant les tableaux de toute autre classe**.

On peut aussi observer **une passerelle avec la DataFrame**, classe bien connue du module pandas : DF.columns affiche les colonnes d'une instance de DataFrame. 

Ce **columns est un attribut d'une instance de DataFrame** de la même façon que notre **l_tableaux est un attribut d'une instance de Wikidaper**.

On va paramétrer une fonction de desciption de l_tableaux, qui va afficher le nombre de tableaux et leurs dimensions quand il existent.

En utilisant BeautifulSoup, ça va être assez simple : 
* côté lignes, il suffira de **sélectionner l'ensemble d'un tableau** (balises <tr></tr>) **et de retrancher** un pour les colonnes
* côté colonnes, ça va être un peu plus coton, notamment pour ce cas de figure :

![col_span](images/col_span.jpg)

Huit colonnes, mais deux lignes... C'est potentiellement casse-pied et il va falloir ruser, mais en gros :
* les noms de colonnes se sélectionnent facilement grâce à la balise th (ou table header). On a juste **à sélectionner celles qui ne possèdent pas d'attribut colspan** (ie qui s'étalent sur plusieurs colonness)
* les balises <tr> (acronyme de table row) qui n'ont pas d'enfant th peuvent être considérées
* on dénombre l'une comme l'autre liste et on n'en parle plus
    
Du coup, nouvelle mise à jour :

In [16]:
class Wikidaper:
    """
        Cette classe sert à aspirer les tableaux HTML qui contiennent des données classables dans une page Wikipedia.
        Les données chiffrées sont converties en entiers ou nombres à décimales et les tableaux restitués en DataFrames.
    """
    def __init__(self,url:str):
        """
            On initialise avec :
                - une URL renseignée par l'utilisateur, de préférence copiée/collée depuis le navigateur
                - un booléen de validation de cette dernière
                - une liste composée des tableaux classables trouvés sur la page
        """
        self.url=url
        self.valid_url=self.valide_url()
        if self.valid_url:
            self.l_tableaux=self.recolte_tableaux()
        else:
            self.l_tableaux=[]
        
    def valide_url(self):
        """
            Cette fonction va vérifier si l'URL rentrée par l'utilisateur renvoie vers une page Wikipedia susceptible d'avoir des tableaux avec données classables.
        """
        result=False
        regex="^http[s]{,1}:[/]{2}[a-z]{2,}\.wikipedia\.org/wiki/[A-Za-z0-9]+[_A-Za-z0-9]*"
        if re.match(regex, self.url):
            try:
                requests.get(self.url)
                result=True
            except:
                print("Cette page Wikipedia ne répond pas")
        else:
            print("Attention, votre URL ne pointe pas vers une page Wikipedia valide")
        return result
    
    def recolte_tableaux(self):
        """
            Cette fonction retourne une liste des tableaux classables trouvables sur une page Wikipedia.
            Elle reste vide s'il n'y a rien.
        """
        l_tableaux=[]
        url=requests.get(self.url)
        soupe=BeautifulSoup(url.text, "lxml")
        regex=re.compile('^wikitable.+sortable.*')
        for t in soupe.find_all("table", {"class" : regex}):
            l_tableaux.append(t)
        return l_tableaux
    
    def describe(self):
        """
            Cette fonction affiche les dimensions de chaque tableau de données classables trouvés dans la page Wikipedia renseignée 
        """
        if len(self.l_tableaux)==0:
            print("Il n'y a aucun tableau avec des données classables sur cette page")
        else:
            print(len(self.l_tableaux)," tableau(x) de données classables sur cette page, de dimension :")
            for t in self.l_tableaux:
                n_col=len([c.text for c in t.find_all("th") if not c.has_attr("colspan")])
                n_lignes=len([l for l in t.find_all("tr") if not l.findChildren("th" , recursive=False)])
                print(n_col," colonnes x",n_lignes,"lignes")

Comme d'habitude, on teste sur une page sans tableau...

In [17]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Psychanalyse")
test_wiki.describe()

Il n'y a aucun tableau avec des données classables sur cette page


...une avec un tableau...

In [18]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg")
test_wiki.describe()

1  tableau(x) de données classables sur cette page, de dimension :
6  colonnes x 33 lignes


...voire plus :

In [19]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées")
test_wiki.describe()

2  tableau(x) de données classables sur cette page, de dimension :
14  colonnes x 283 lignes
16  colonnes x 25 lignes


Les dimensions sont parfaitement conformes à ce qu'on observe sur les pages ! On va bientôt pouvoir attaquer le gros morceau, mais nous allons coder une fonction de conversion avant.

Cette dernière servira **à transformer des données chiffrées au format anglo-saxon**, en prenant en unique attribut une chaîne de caractères. Elle sera notamment comparé à deux expressions régulières : 
* la première **au format décimal français** (série de chiffres séparés par une virgule)
* la seconde se plaie **aux chiffres entiers français** (milliers séparés par des espaces)

Et sinon, la chaîne demeure telle quelle. Précision importante : on ne convertit pas dans la fonction les valeurs changées, et tenter les conversions par colonne une fois les DataFrames créées.

In [20]:
def convertinoat(chaine:str):
    """
        Cette fonction convertit, si c'est possible, une chaîne chiffrée à la française (espace entre les milliers, virgules pour séparer des décimales) à un format anglo-saxon.
    """
    chaine=chaine.split(" (")[0]
    chaine=chaine.split("[")[0]
    chaine=chaine.replace("\n","")
    chaine=chaine.replace(u'\u200d', u'')
    chaine=chaine.replace(u'\xa0', u' ')
    reg_float=r"^[0-9]{1,3}( [0-9]{3})*,[0-9]+$"
    reg_int=r"^[0-9]{1,3}( [0-9]{3})*$"
    if re.match(reg_float,chaine):
        chaine=chaine.replace(" ","")
        chaine=chaine.replace(",",".")
    elif re.match(reg_int,chaine):
        chaine=chaine.replace(" ","")
    return chaine

Quelques tests, juste pour l'hygiène :

In [21]:
print(convertinoat("5 256,65897"))
print(convertinoat("5 256,65897%"))
print(convertinoat("98 210 356 879"))
print(convertinoat("9812 210 356 879"))

5256.65897
5 256,65897%
98210356879
9812 210 356 879


Et maintenant le gros morceau : constituer une DataFrame à partir d'un des tableaux. Trois grandes étapes :

* **vérifier que l'indice** saisi par l'utilisateur est bien inclus dans les bornes de la liste l_tableaux
* **créer une liste de dictionnaires**. Dès le départ, on récupére les noms de colonnes dans une liste puis, à chaque ligne du tableau hors colonnes, on compte son nombre de cellules. S'il est égal au nombre de colonnes tout roule, on peut faire une compréhension de dictionnaires à partir de la liste de colonnes : la première cellule de la ligne considérée correspondra bien à la première colonne, etc... **Si ce nombre de cellules diffère, on un colspan qui traîne** et on va faire radical et doublonner autant de fois que nécessaire dans une liste dédiée. Le plus important est que la taille des lignes soit bien conforme à celle de la liste de colonnes. Une fois la liste de dictionnaire complète, on la transforme en DataFrame grâce [à la fonction pandas DataFrame.from_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html)
* et ensuite, on parcourt les colonnes. Si la colonne n'apparaît pas dans une liste d'exceptions remplie à l'appel de la fonction, on essaye de la convertir en nombres entiers. Si ça plante à cause d'une ValueError, on essaye de la convertir en nombres à décimales. Si ça plante encore à cause d'une ValueError, on passe à la suite. Cela se fait très facilement [avec des try/except](https://docs.python.org/3/tutorial/errors.html)

In [22]:
class Wikidaper:
    """
        Cette classe sert à aspirer les tableaux HTML qui contiennent des données classables dans une page Wikipedia.
        Les données chiffrées sont converties en entiers ou nombres à décimales et les tableaux restitués en DataFrames.
    """
    def __init__(self,url:str):
        """
            On initialise avec :
                - une URL renseignée par l'utilisateur, de préférence copiée/collée depuis le navigateur
                - un booléen de validation de cette dernière
                - une liste composée des tableaux classables trouvés sur la page
        """
        self.url=url
        self.valid_url=self.valide_url()
        if self.valid_url:
            self.l_tableaux=self.recolte_tableaux()
        else:
            self.l_tableaux=[]
        
    def valide_url(self):
        """
            Cette fonction va vérifier si l'URL rentrée par l'utilisateur renvoie vers une page Wikipedia susceptible d'avoir des tableaux avec données classables.
        """
        result=False
        regex="^http[s]{,1}:[/]{2}[a-z]{2,}\.wikipedia\.org/wiki/[A-Za-z0-9]+[_A-Za-z0-9]*"
        if re.match(regex, self.url):
            try:
                requests.get(self.url)
                result=True
            except:
                print("Cette page Wikipedia ne répond pas")
        else:
            print("Attention, votre URL ne pointe pas vers une page Wikipedia valide")
        return result
    
    def recolte_tableaux(self):
        """
            Cette fonction retourne une liste des tableaux classables trouvables sur une page Wikipedia.
            Elle reste vide s'il n'y a rien.
        """
        l_tableaux=[]
        url=requests.get(self.url)
        soupe=BeautifulSoup(url.text, "lxml")
        regex=re.compile('^wikitable.+sortable.*')
        for t in soupe.find_all("table", {"class" : regex}):
            l_tableaux.append(t)
        return l_tableaux
    
    def describe(self):
        """
            Cette fonction affiche les dimensions de chaque tableau de données classables trouvés dans la page Wikipedia renseignée 
        """
        if len(self.l_tableaux)==0:
            print("Il n'y a aucun tableau avec des données classables sur cette page")
        else:
            print(len(self.l_tableaux)," tableau(x) de données classables sur cette page, de dimension :")
            for t in self.l_tableaux:
                n_col=len([c.text for c in t.find_all("th") if not c.has_attr("colspan")])
                n_lignes=len([l for l in t.find_all("tr") if not l.findChildren("th" , recursive=False)])
                print(n_col," colonnes x",n_lignes,"lignes")
    
    def df_table(self,indice:int,l_except:list):
        """
            Cette fonction renvoie une DataFrame à partir d'un tableau listé dans l_tableaux.
        """
        indice=abs(indice)
        if len(self.l_tableaux)==0:
            print("Il n'y a aucun tableau avec des données classables sur cette page")
        elif indice>(len(self.l_tableaux)-1):
            print("Attention, vous devez interroger un indice compris entre 0 et ",(len(self.l_tableaux)-1))
        else:
            print("Transformation de la table ",indice,"en cours")
            l_dico=[]
            l_col=[c.text.split("[")[0].replace("\n","") for c in self.l_tableaux[indice].find_all("th") if not c.has_attr("colspan")]
            print("Colonnes trouvées :",l_col)
            for l in self.l_tableaux[indice].find_all("tr"): 
                if not l.findChildren("th" , recursive=False):
                    if len(l.find_all("td"))==len(l_col):
                        l_dico.append({l_col[i]:convertinoat(l.find_all("td")[i].get_text()) for i in range(len(l_col))})
                    else:
                        l_rattrapage=[]
                        for e in l.find_all("td"):
                            if e.has_attr("colspan"):
                                doublon=convertinoat(e.get_text())
                                iterations=int(e.get("colspan"))
                                for i in range(iterations):
                                    l_rattrapage.append(doublon)
                            else:
                                l_rattrapage.append(convertinoat(e.get_text()))
                        l_dico.append({l_col[i]:l_rattrapage[i] for i in range(len(l_col))})
            DF=pd.DataFrame.from_dict(l_dico)
            for c in DF.columns:
                if c not in l_except:
                    try:
                        DF[c]=DF[c].astype(int)
                    except ValueError:
                        try:
                            DF[c]=DF[c].astype(float)
                        except ValueError:
                            pass
            return DF

On va attaquer un gros morceau avec la première liste des communes de France les plus peuplées, en mettant bien dans une exception la colonne "CodeInsee" pour qu'elle ne soit pas convertie en chiffres :

In [23]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es")
DF=test_wiki.df_table(0,["CodeInsee"])
DF.tail()

Transformation de la table  0 en cours
Colonnes trouvées : ['Rang2022', 'CodeInsee', 'Commune', 'Département', 'Statut', 'Région', '2019', '2013', '2008', '1999', '1990', '1982', '1975', '1968']


,Rang2022,CodeInsee,Commune,Département,Statut,Région,2019,2013,2008,1999,1990,1982,1975,1968
278,275,78146,Chatou,Yvelines,—,Île-de-France,30153,30809,29940,28588,27977,28437,26550,22619
279,276,62510,Liévin,Pas-de-Calais,—,Hauts-de-France,30112,31517,32026,33427,33623,33096,33070,35853
280,277,93063,Romainville,Seine-Saint-Denis,—,Île-de-France,30087,25657,25621,23779,23563,25363,26260,24091
281,278,92060,Le Plessis-Robinson,Hauts-de-Seine,—,Île-de-France,30061,28500,24675,21618,21289,21271,22231,22590
282,279,92064,Saint-Cloud,Hauts-de-Seine,—,Île-de-France,30012,29109,29772,28157,28597,28561,28139,28158


Ca semble pas mal, mais lançons-nous dans quelques vérifs. Par exemple en extrayant le code INSEE de Nice :

In [24]:
DF[DF["Commune"]=="Nice"]["CodeInsee"]

4    06088
Name: CodeInsee, dtype: object

Il commence bien par un "0", ce qui prouve que **l'exception précisée en appel de fonction a été prise en compte**. Essayons maintenant de soustraire le nombre d'habitants de Paris en 1990 à celui de 2019 :

In [25]:
(DF[DF["Commune"]=="Paris"]["2019"].iloc[0])-(DF[DF["Commune"]=="Paris"]["1990"].iloc[0])

13000

Seulement 13 000 habitants de plus en 29 ans. Il y a quelques exceptions, par exemple **les colonnes 1975 et 1968 sont encore formatées en chaînes de caractères** en raison de recensements manquants pour les villes de Mayotte.

Toujours est-il que, grâce à la classe Wikidaper, on peut arriver à un résultat tangible pour l'Eurométropole de Strasbourg :

In [26]:
test_wiki=Wikidaper("https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg")
DF=test_wiki.df_table(0,["Code  Insee"])
DF["Population  (dernière pop. légale)"].sum()

Transformation de la table  0 en cours
Colonnes trouvées : ['Nom', 'Code  Insee', 'Gentilé', 'Superficie  (km2)', 'Population  (dernière pop. légale)', 'Densité  (hab./km2)']


505272

505 272 habitants dans l'Eurométropole, contrairement à l'erreur renvoyée en début de calepin. On peut aussi calculer la médiane de superficie :

In [27]:
DF["Superficie  (km2)"].median()

6.25

 De beaux chiffres à décimales, et concernant la liste de codes INSEE, une nouvelle variable fera l'affaire :

In [28]:
insee_eurom=list(DF["Code  Insee"])
insee_eurom[:10]

['67482',
 '67001',
 '67043',
 '67049',
 '67065',
 '67118',
 '67119',
 '67124',
 '67131',
 '67137']

# Derniers fignolages

Un cas intéressant mais qui ne concerne pas les données classables dans Wikipedia est le suivant :

![pivot_wiki](images/wiki_pivot.jpg)

Des tableaux d'une ligne et n colonnes qui s'empilent, que l'on pourrait très bien transformer afin d'obtenir une matrice de deux colonnes et n lignes. Comment faire ? D'abord, partons de ces deux lignes :

In [29]:
paris_demo=pd.read_html('https://fr.wikipedia.org/wiki/Paris', match="Évolution de la population de Paris depuis l'Antiquité")
paris_demo[5].head()

,1982,1990,1999,2008,2013,2019,-,-.1,-.2
0,2 176 243,2 152 423,2 125 246[314],2 211 297[315],2 229 621[316],2 165 423[317],-,-,-


read_html a renvoyé une liste dont chaque indice pointe vers une DataFrame. Il y en a six dans cette liste, comme les six lignes de dates relevées sur Wikipedia.

On peut utiliser cette liste en :
* prenant en référence **la DataFrame située à l'indice 0**
* si la liste contient plus d'éléments, on ajoute **chaque colonne de la nouvelle DataFrame dans la première

**On obtient alors une seule DataFrame de n colonnes**, et on peut boucler sur ces n colonnes pour créer une liste de dictionnaires à deux items, dont les clés sont renseignées par l'utilisateur. Codage, et démo :

In [30]:
def pivot_wiki(url:str,n_tableau:str,l_colonnes:list):
    """
        Cette fonction fait pivoter à la verticale des tableaux Wikipedia à une ligne et x colonnes pour la transformer en DataFrame de deux colonnes sur x lignes.
    """
    l_finale=[]
    l_DF=pd.read_html(url, match=n_tableau)
    DF=l_DF[0].copy()
    if len(l_DF)>1:
        for i in range(1,len(l_DF)):
            DF_trans=l_DF[i]
            for c in DF_trans.columns:
                DF[c]=DF_trans[c]
    for f_c in DF:
        l_finale.append({l_colonnes[0]:f_c,l_colonnes[1]:convertinoat(DF[f_c].iloc[0])})
    DF_finale=pd.DataFrame.from_dict(l_finale)
    return DF_finale

Note : on peut appeler la fonction convertinoat dans ces conditions car elle a été codées hors de la classe Wikidaper.

À présent, on va tester ce que l'on vient de coder, en espérant avoir à l'arrivée une DataFrame à deux colonnes (annee et nb) :

In [31]:
demo_paris=pivot_wiki("https://fr.wikipedia.org/wiki/Paris","Évolution de la population de Paris depuis l'Antiquité",["annee","nb"])
demo_paris.tail()

,annee,nb
49,2013,2229621
50,2019,2165423
51,-,-
52,-.1,-
53,-.2,-


Les trois dernières valeurs sont bloquantes pour la conversion en nombres entiers, on va donc les virer, convertir la colonne "nb", et vérifier si ça a marché en affichant l'année à laquelle la ville de Paris a été la plus peuplée :

In [32]:
demo_paris=demo_paris.iloc[:50]
demo_paris["nb"]=demo_paris["nb"].astype(int)
demo_paris[demo_paris["nb"]==max(demo_paris["nb"])]["annee"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


36    1921
Name: annee, dtype: object

On n'a donc jamais fait plus qu'en 1921, ce qui est bien conforme aux différents tableaux vus sur la page Wikipedia.

# Import du module maison

Une fois [le code bien hébergé sur Github](https://github.com/raphadasilva/wikidaper), on peut le réutiliser facilement via la commande magique !git

NB : j'ai un peu triché en important ce calepin local sur Google Colab, mais il doit sûrerment y avoir une bidouille pour faire tourner correctement tout ça.

In [33]:
!git clone https://github.com/raphadasilva/wikidaper.git

Cloning into 'wikidaper'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


Une fois le repo cloné, on peut **appeler le fichier wikidaper comme un module à part entière**, avec son acronyme qui va bien et ses fonction qui tournent comme souhaitées :

In [34]:
import wikidaper.wikidaper as wd
test_wiki=wd.Wikidaper("https://fr.wikipedia.org/wiki/Eurom%C3%A9tropole_de_Strasbourg")
DF=test_wiki.df_table(0,["Code  Insee"])
DF["Population  (dernière pop. légale)"].sum()

Transformation de la table  0 en cours
Colonnes trouvées : ['Nom', 'Code  Insee', 'Gentilé', 'Superficie  (km2)', 'Population  (dernière pop. légale)', 'Densité  (hab./km2)']


505272